In [1]:
import json
import numpy as np
import pandas as pd

load('utils.sage')

### Compute vanishing ideal of tau(S)

In [2]:
GammaAdjMatrix = np.array([[1, 0],
                           [1, 0],
                           [1, 0],
                           [1, 1],
                           [0, 1],
                           [0, 1]])

I_Omega, basis_Omega = get_vanishing_ideal_Omega(GammaAdjMatrix)
basis_Omega

[w12*w34 - w14*w23, w13*w24 - w14*w23, w15, w16, w25, w26, w35, w36]

### Compute polynomials psi(lambda, sigma)

In [3]:
n = 26
order = 'deglex(1),deglex(4),deglex(21)' # block-monomial order
R.<xi,l12,l23,l45,l46, s11,s12,s13,s14,s15,s16,s22,s23,s24,s25,s26,s33,s34,s35,s36,s44,s45,s46,s55,s56,s66> = PolynomialRing(QQ, n, order=order)
R.inject_variables()

Defining xi, l12, l23, l45, l46, s11, s12, s13, s14, s15, s16, s22, s23, s24, s25, s26, s33, s34, s35, s36, s44, s45, s46, s55, s56, s66


In [4]:
Lambda = matrix([[0, l12, 0, 0, 0, 0],
                 [0, 0, l23, 0, 0, 0], 
                 [0, 0, 0, 0, 0, 0], 
                 [0, 0, 0, 0, l45, l46], 
                 [0, 0, 0, 0, 0, 0], 
                 [0, 0, 0, 0, 0, 0]])
Lambda

[  0 l12   0   0   0   0]
[  0   0 l23   0   0   0]
[  0   0   0   0   0   0]
[  0   0   0   0 l45 l46]
[  0   0   0   0   0   0]
[  0   0   0   0   0   0]

In [5]:
Sigma = matrix([[s11,s12,s13,s14,s15,s16],
                [s12,s22,s23,s24,s25,s26],
                [s13,s23,s33,s34,s35,s36],
                [s14,s24,s34,s44,s45,s46],
                [s15,s25,s35,s45,s55,s56],
                [s16,s26,s36,s46,s56,s66]])
Sigma

[s11 s12 s13 s14 s15 s16]
[s12 s22 s23 s24 s25 s26]
[s13 s23 s33 s34 s35 s36]
[s14 s24 s34 s44 s45 s46]
[s15 s25 s35 s45 s55 s56]
[s16 s26 s36 s46 s56 s66]

In [6]:
Id = identity_matrix(6)
Id

[1 0 0 0 0 0]
[0 1 0 0 0 0]
[0 0 1 0 0 0]
[0 0 0 1 0 0]
[0 0 0 0 1 0]
[0 0 0 0 0 1]

In [7]:
psi = (Id-Lambda).T * Sigma * (Id-Lambda)

### Compute final Gröbner basis

In [8]:
generators = get_constraints(psi, basis_Omega)
generators.append(xi * det((Id-Lambda))-1)
generators

[l12*l23*s11*s24 - l12*l23*s12*s14 - l12*s11*s34 + l12*s13*s14 - l23*s12*s24 + l23*s14*s22 + s12*s34 - s14*s23,
 -l23*s12*s24 + l23*s14*s22 + s13*s24 - s14*s23,
 -l45*s14 + s15,
 -l46*s14 + s16,
 l12*l45*s14 - l12*s15 - l45*s24 + s25,
 l12*l46*s14 - l12*s16 - l46*s24 + s26,
 l23*l45*s24 - l23*s25 - l45*s34 + s35,
 l23*l46*s24 - l23*s26 - l46*s34 + s36,
 xi - 1]

In [9]:
I = R.ideal(generators)
I = I.elimination_ideal([xi]) 
basis = I.groebner_basis(algorithm="libsingular:groebner")
basis

Polynomial Sequence with 33 Polynomials in 18 Variables

In [10]:
def is_identifiable(param,direct_effects,basis,R):
    for i in range(0,len(basis)):
        lm = basis[i].lm()
        cond1 = R.monomial_divides(param, lm)
        cond2 = (len(set(R.monomial_quotient(lm, param).variables()).intersection(set(direct_effects))) == 0)
        if cond1 and cond2:
            return({f"{param} identifiable": True, "basis-polynomial": basis[i]})
    return({f"{param} identifiable": False})

In [11]:
direct_effects = [l12,l23,l45,l46]

In [12]:
is_identifiable(l12, direct_effects, basis,R)

{'l12 identifiable': True,
 'basis-polynomial': l12*s11*s12*s24*s34 - l12*s11*s13*s24^2 - l12*s11*s14*s22*s34 + l12*s11*s14*s23*s24 - l12*s12*s14^2*s23 + l12*s13*s14^2*s22 - s12^2*s24*s34 + s12*s13*s24^2 + s12*s14*s22*s34 - s13*s14*s22*s24}

In [13]:
is_identifiable(l23, direct_effects, basis,R)

{'l23 identifiable': True,
 'basis-polynomial': l23*s12*s24 - l23*s14*s22 - s13*s24 + s14*s23}

In [14]:
is_identifiable(l45, direct_effects, basis,R)

{'l45 identifiable': True, 'basis-polynomial': l45*s14 - s15}

In [15]:
is_identifiable(l46, direct_effects, basis,R)

{'l46 identifiable': True, 'basis-polynomial': l46*s14 - s16}